In [44]:
import pandas as pd

In [45]:
df = pd.read_csv("./unprepared_df.csv")

df.head()

,Unnamed: 0,Year,Sale Date,Sale Price,Property Location,Year Built,Garden,Garage,Type,Style,Bedrooms,Bathrooms,First Time Buyer
0,0,2021,Sold 2021-01-10,"€381,302.00",Broomhouse,1967,Yes,Yes,Detached,1.5-Storey,3,1,No
1,1,2021,Sold 2021-01-10,"€325,898.00",Broomhouse,1978,Yes,???,Detached,1-Storey,3,1,Yes
2,2,2021,Sold 18 January 2021,"€ 370,354",Oak Park,1961,Yes,No,Detached,1-Storey,3,2,No
3,3,2021,Sold 2021-01-23,"€92,480.00",Beacon Hill,1958,Yes,No,Bungalow,1-Storey,1,1,Yes
4,4,2021,Sold 2021-01-25,"€312,030.00",Brookville,1987,Yes,Yes,Detached,1-Storey,3,1,No


In [46]:
df['Sale Date'] = df['Sale Date'].str.slice(start=4, stop=None)
df['Sale Date'] = pd.to_datetime(df['Sale Date'], format="mixed")

df['Sale Price'] = df['Sale Price'].str.slice(start=1, stop=None)
df['Sale Price'] = df['Sale Price'].str.replace(',','')

df['Year Built'] = df['Year Built'].str.replace('c','')
df['Year Built'] = df['Year Built'].str.replace('Unknown', '0')

df['Style'] = df['Style'].str.replace('-Storey', '')


In [47]:
df['Sale Price'] = df['Sale Price'].astype('float')
df['Property Location'] = df['Property Location'].astype('category')
df['Year Built'] = df['Year Built'].astype('int')
df['Garden'] = df['Garden'].map({"Yes": True, "No": False})
df['Garden'] = df['Garden'].astype('bool')
df['Garage'] = df['Garage'].map({"Yes": True, "No": False})
df['Garage'] = df['Garage'].astype('bool')
df['Type'] = df['Type'].astype('category')
df['Style'] = df['Style'].astype('category')
df['Bedrooms'] = df['Bedrooms'].astype('int')
df['Bathrooms'] = df['Bathrooms'].astype('int')
df['First Time Buyer'] = df['First Time Buyer'].map({'Yes': True, 'No': False})
df['First Time Buyer'] = df['First Time Buyer'].astype('bool')

In [48]:
df = df.drop(columns=["Unnamed: 0", "Year"])

df.head()

,Sale Date,Sale Price,Property Location,Year Built,Garden,Garage,Type,Style,Bedrooms,Bathrooms,First Time Buyer
0,2021-01-10,381302.0,Broomhouse,1967,True,True,Detached,1.5,3,1,False
1,2021-01-10,325898.0,Broomhouse,1978,True,True,Detached,1,3,1,True
2,2021-01-18,370354.0,Oak Park,1961,True,False,Detached,1,3,2,False
3,2021-01-23,92480.0,Beacon Hill,1958,True,False,Bungalow,1,1,1,True
4,2021-01-25,312030.0,Brookville,1987,True,True,Detached,1,3,1,False


In [49]:
df.dtypes

Sale Date            datetime64[ns]
Sale Price                  float64
Property Location          category
Year Built                    int64
Garden                         bool
Garage                         bool
Type                       category
Style                      category
Bedrooms                      int64
Bathrooms                     int64
First Time Buyer               bool
dtype: object

In [50]:
df.to_csv('./prepared_df.csv')